In [ ]:
import numpy as np
import cv2
#print(cv2.__version__)
 
def onTrack1(val):
    global hueLow
    hueLow=val
    #print('Hue Low',hueLow)
def onTrack2(val):
    global hueHigh
    hueHigh=val
    #print('Hue High',hueHigh)
def onTrack3(val):
    global satLow
    satLow=val
    #print('Sat Low',satLow)
def onTrack4(val):
    global satHigh
    satHigh=val
    #print('Sat High',satHigh)
def onTrack5(val):
    global valLow
    valLow=val
    #print('Val Low',valLow)
def onTrack6(val):
    global valHigh
    valHigh=val
    #print('Val High',valHigh)
 
width=640
height=360
cam=cv2.VideoCapture(0,cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT,height)
cam.set(cv2.CAP_PROP_FPS, 30)
cam.set(cv2.CAP_PROP_FOURCC,cv2.VideoWriter_fourcc(*'MJPG'))
 
cv2.namedWindow('myTracker')
cv2.moveWindow('myTracker',width,0)
 
hueLow=0
hueHigh=179
satLow=0
satHigh=255
valLow=0
valHigh=255
 
cv2.createTrackbar('Hue Low','myTracker',0,179,onTrack1)
cv2.createTrackbar('Hue High','myTracker',179,179,onTrack2)
cv2.createTrackbar('Sat Low','myTracker',0,255,onTrack3)
cv2.createTrackbar('Sat High','myTracker',255,255,onTrack4)
cv2.createTrackbar('Val Low','myTracker',0,255,onTrack5)
cv2.createTrackbar('Val High','myTracker',255,255,onTrack6)
 
while True:
    # ignore,  frame = cam.read()
    frame = cv2.imread("1.jpg")
    frameHSV=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    lowerBound=np.array([hueLow,satLow,valLow])
    upperBound=np.array([hueHigh,satHigh,valHigh])
    myMask=cv2.inRange(frameHSV,lowerBound,upperBound)
    #myMask=cv2.bitwise_not(myMask)
    myObject=cv2.bitwise_and(frame,frame,mask=myMask)
    myObjectSmall=cv2.resize(myObject,(int(width/2),int(height/2)))
    cv2.imshow('My Object',myObjectSmall)
    cv2.moveWindow('My Object',int(width/2),int(height))
    myMaskSmall=cv2.resize(myMask,(int(width/2),int(height/2)))
    cv2.imshow('My Mask',myMaskSmall)
    cv2.moveWindow('My Mask',0,height)
    cv2.imshow('my WEBcam', frame)
    cv2.moveWindow('my WEBcam',0,0)
    if cv2.waitKey(1) & 0xff ==ord('q'):
        break
cam.release()

In [3]:
from collections import deque
import numpy as np
import cv2
import imutils


BUFFER_SIZE = 64
# orange color range
COLOR_LOWER = (12, 114, 191)
COLOR_UPPER = (152, 255, 255)

pts = deque(maxlen=BUFFER_SIZE)


In [4]:
vs = cv2.VideoCapture(0,cv2.CAP_DSHOW)

while True:
    ret, frame = vs.read()

    if not ret or frame is None:
        break

    frame = imutils.resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, COLOR_LOWER, COLOR_UPPER)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None

    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        if radius > 5:
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
            diameter = radius * 2
            cv2.putText(frame, "Diameter: {:.2f}".format(diameter), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    pts.appendleft(center)

    for i in range(1, len(pts)):
        if pts[i - 1] is None or pts[i] is None:
            continue
        thickness = int(np.sqrt(BUFFER_SIZE / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(10) & 0xff ==ord('q'):
        break

vs.release()
cv2.destroyAllWindows()